In [1]:
print("all ok")

all ok


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["GOOGLE_API_KEY"]=os.getenv("GOOGLE_API_KEY")
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["LANGSMITH_TRACING"]=os.getenv("LANGSMITH_TRACING")
os.environ["LANGSMITH_ENDPOINT"]=os.getenv("LANGSMITH_ENDPOINT")
os.environ["LANGSMITH_API_KEY"]=os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"]=os.getenv("LANGSMITH_PROJECT")
os.environ["QROQ_API_KEY"]= os.getenv("QROQ_API_KEY")
os.environ["HF_API_TOKEN"]=os.getenv("HF_API_TOKEN")

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
embedding_model=HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')

In [5]:
len(embedding_model.embed_query("Hello how are you"))

768

In [6]:
from langchain_openai.embeddings import OpenAIEmbeddings

openai_embedding=OpenAIEmbeddings( model = "text-embedding-ada-002")
len(openai_embedding.embed_query("Hello how are you"))

1536

In [17]:
from langchain.prompts import PromptTemplate

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm=ChatGoogleGenerativeAI(model='gemini-2.5-flash')

In [15]:
template="Hi , I am learning {skill} . can you suggest me top 5 things I should learn?"

In [16]:
prompt=PromptTemplate(template=template,input_variables=['skill'])

### LCEL Chain concept

In [8]:
template="Hi , I am learning {skill} . can you suggest me top 5 things I should learn?"

In [9]:
from langchain.prompts import PromptTemplate

In [10]:
prompt=PromptTemplate(template=template,input_variables=['skill'])

In [11]:
chain=prompt | llm

In [12]:
chain.invoke({"skill": "data science"})

AIMessage(content="That's a great question! Data science is a vast field, but there are definitely core pillars that will give you a strong foundation. For someone starting out, here are my top 5 suggestions:\n\n---\n\n### Top 5 Things to Learn for Data Science Beginners:\n\n1.  **Programming (Primarily Python or R, with a strong lean towards Python)**\n    *   **Why it's crucial:** This is your primary tool for manipulating data, building models, and automating tasks. Python is widely adopted in the industry due to its versatility, extensive libraries, and ease of learning. R is also excellent, especially for statistical analysis and visualization.\n    *   **What to focus on:**\n        *   **Python Fundamentals:** Variables, data types, control structures (if/else, loops), functions, object-oriented programming basics.\n        *   **Key Libraries:**\n            *   **Pandas:** For data manipulation and analysis (e.g., reading files, cleaning data, merging dataframes, aggregation).

In [13]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()

chain=prompt | llm | parser

In [14]:
chain.invoke({"skill": "data science"})

'That\'s fantastic! Data science is a dynamic and rewarding field. To give you the best foundational and practical skills, here are my top 5 things you should learn:\n\n---\n\n### Top 5 Things to Learn in Data Science:\n\n1.  **Programming (Python/R & SQL)**\n    *   **Why it\'s crucial:** These are the languages you\'ll use to manipulate data, build models, and automate tasks. Python is widely popular for its versatility (data analysis, machine learning, web development) and rich ecosystem of libraries (Pandas, NumPy, Scikit-learn). R is excellent for statistical analysis and visualization. SQL is essential for interacting with databases, which is where most real-world data lives.\n    *   **Key things to learn:**\n        *   **Python:** Data structures (lists, dictionaries), control flow, functions, object-oriented programming basics.\n        *   **Key Libraries (Python):** `Pandas` (data manipulation), `NumPy` (numerical operations), `Matplotlib`/`Seaborn` (visualization), `Scikit

### Introduce Runnable methods

In [20]:
## RunnablePassThrough- pass input at the time of run time

from langchain_core.runnables import RunnablePassthrough,RunnableParallel,RunnableLambda

In [21]:
chain=RunnablePassthrough()

In [22]:
chain.invoke("Hello Welcome to Kolkata")

'Hello Welcome to Kolkata'

In [23]:
# RunnableLambda() used to execute any function on the input value

def make_upper(text):
    return text.upper()

In [25]:
chain=RunnablePassthrough() | RunnableLambda(make_upper)

In [26]:
chain.invoke("Hello Welcome to Kolkata")

'HELLO WELCOME TO KOLKATA'

In [27]:
## RunnableParellel() used to execute multiple input in parallel

chain= RunnableParallel({'x':RunnablePassthrough(),'y': RunnablePassthrough()})

In [28]:
chain.invoke("Hello Tapas")

{'x': 'Hello Tapas', 'y': 'Hello Tapas'}

In [29]:
chain= RunnableParallel({'x':RunnablePassthrough(),'blog': lambda x: x['blog']})

In [30]:
chain.invoke({'youtube':'campasx','blog':'campasx"s blog'})

{'x': {'youtube': 'campasx', 'blog': 'campasx"s blog'},
 'blog': 'campasx"s blog'}

In [31]:
def fetch_website(text:dict):
    output=text.get('website','Not Found')
    return output

In [32]:
chain=RunnableParallel({'website': RunnablePassthrough() | RunnableLambda(fetch_website),
                        'blog':lambda z:z['blog']})

In [33]:
chain.invoke({'youtube':'campasx','blog':'campasx"s blog'})

{'website': 'Not Found', 'blog': 'campasx"s blog'}

In [34]:
chain.invoke({'youtube':'campasx','blog':'campasx"s blog','website':'campasx.com'})

{'website': 'campasx.com', 'blog': 'campasx"s blog'}

In [38]:
def add_extra(input):
    return 'Happy Learning'

In [39]:
chain=RunnableParallel({'x': RunnablePassthrough()}).assign(y=RunnableLambda(add_extra))

In [40]:
chain.invoke('hello')

{'x': 'hello', 'y': 'Happy Learning'}

## Building RAG and different calling method

In [41]:
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_core.output_parsers import StrOutputParser

In [42]:
loader=TextLoader(r'F:\Tapas\Learning\GenAI\data\state_of_the_union.txt')
doc=loader.load()

In [43]:
len(doc)

1

In [46]:
spliter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
docs=spliter.split_documents(doc)

In [47]:
len(docs)

91

In [50]:
vectore_store=Chroma(collection_name='collection',embedding_function=openai_embedding)

In [51]:
vectore_store.add_documents(docs)

['6759ebcf-15ec-4bac-93fe-8e4fa989b193',
 '358c9410-56c2-48a9-8a68-55bde2df0c2f',
 '83bdd429-30b1-4a16-b0ef-8204b7450918',
 '9c46bc8f-9b4e-4439-919a-3f1885bfc509',
 '338a613a-d649-4f06-96b9-53e3b4c592ac',
 '428dee02-a257-48dc-b82f-dc99004442fe',
 '8269ed37-6352-4f4b-8e3d-cf522e4115ad',
 '1a755e6f-cd7f-4c57-9bcb-02a8c39830b6',
 '51f65f6b-5945-4e20-b2cc-11e45907a8f9',
 '790c5ed0-34e0-4aee-823e-01fd0254e70e',
 '8048a14c-f516-40a0-a3da-53864d672f53',
 'f11b7531-a76d-4c17-8586-8dd9b52e96e4',
 '39360b4b-b167-4a08-9352-fde16b3b720c',
 '4b79c38a-81e6-40b4-a165-9ef1ffebd908',
 '55f3f71a-ef82-4f3a-8b38-b54148136513',
 '557bfc27-d277-4f56-b88e-2784cbb5eae8',
 '7a565f6a-9e03-4b50-8318-796c2dc72022',
 '7743a45e-e83b-45f4-8462-5fbf55b01a2e',
 '74fc329f-6ce5-47be-a5f0-f265a8e41a58',
 'c57609c0-515b-425e-8f12-6087151a48be',
 '62ad7ad8-6e39-48e4-9f3b-a4590f53de94',
 '9f02be67-6526-4bd5-9935-f481e78c4e88',
 '17734b25-e28b-4aaf-9a12-d9788fd4ce6b',
 '9d5327cc-ccc2-4c68-8f0b-187b3a36da12',
 '09ef13a7-2343-

In [52]:
retriever=vectore_store.as_retriever()

In [53]:
template="""
Answer the following question based on the context.If you don't know tel that you don't know.

Context: {context}

Question: {question}
"""
prompt=PromptTemplate(template=template)

In [54]:
retriever_chain=(
    RunnableParallel({"context": retriever, 'question': RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
)

In [56]:
response=retriever_chain.invoke("USA stands on Ukraine")

In [57]:
response

'Yes, the United States of America stands with the Ukrainian people. The context explicitly states: "Yes, we the United States of America stand with the Ukrainian people" and "To our fellow Ukrainian Americans who forge a deep bond that connects our two nations we stand with you."'

In [62]:
import time
start_time=time.time()

response=retriever_chain.invoke("USA stands on Ukraine")
print(response)
print("Time taken: ", time.time()-start_time)

Yes, the United States of America stands with the Ukrainian people. The context explicitly states: "Yes, we the United States of America stand with the Ukrainian people" and "To our fellow Ukrainian Americans who forge a deep bond that connects our two nations we stand with you."
Time taken:  5.448834419250488


In [65]:
start_time=time.time()

result=await retriever_chain.ainvoke("USA stands on Ukraine")
print(result)
print("Time taken: ", time.time()-start_time)

Yes, the United States of America stands with the Ukrainian people.
Time taken:  3.4393131732940674


In [67]:
start_time=time.time()

response=retriever_chain.batch(["USA stands on Ukraine","What is Russia's comment on Ukraine?"])
print(response)
print("Time taken: ", time.time()-start_time)

['Yes, the United States of America stands with the Ukrainian people. The context states, "Yes, we the United States of America stand with the Ukrainian people," and "To our fellow Ukrainian Americans who forge a deep bond that connects our two nations we stand with you."', "I don't know, as the provided context describes actions and interpretations related to Russia's involvement in Ukraine, but it does not include any direct comments made by Russia about Ukraine."]
Time taken:  6.856029748916626


In [68]:
start_time=time.time()

response=await retriever_chain.abatch(["USA stands on Ukraine","What is Russia's comment on Ukraine?"])
print(response)
print("Time taken: ", time.time()-start_time)

['Yes, the United States of America stands with the Ukrainian people. The context explicitly states, "Yes, we the United States of America stand with the Ukrainian people" and "we stand with you" referring to Ukrainian Americans and the Ukrainian people.', "Based on the provided context, there is no direct comment from Russia on Ukraine. The documents describe Russia's actions (unleashing violence and chaos, seeking to shake the free world's foundations), the perceived thoughts/miscalculations of Vladimir Putin regarding Ukraine, and the international response to these actions."]
Time taken:  7.127108573913574


In [71]:
for res in retriever_chain.stream("Usa on Ukraine?"):
    print(res,end='')

Based on the context, the USA's position on Ukraine is one of strong support and solidarity.

*   The US stands with Ukraine, acknowledging the deep bond between the two nations.
*   It sends an "unmistakable signal to Ukraine and to the world" in support.
*   It views the conflict as a "battle between democracy and autocracy" and believes the world is choosing "the side of peace and security."
*   The US draws inspiration from the "iron will," "fearlessness, courage, and determination" of the Ukrainian people.
*   It believes that while Putin may make battlefield gains, he will pay a high price, and he will never gain the hearts and souls of the Ukrainian people, having badly miscalculated their strength.

### Tool binding with the LLM

In [72]:
from langchain_core.tools import tool

In [73]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply a and b."""
    return a * b

In [74]:
model_with_tools=llm.bind_tools([multiply])

In [76]:
result=model_with_tools.invoke("What is 57 multiply by 5")

In [77]:
result.tool_calls

[{'name': 'multiply',
  'args': {'b': 5.0, 'a': 57.0},
  'id': '65eff2b0-e050-4aef-8b6b-309a90b6ae32',
  'type': 'tool_call'}]